In [ ]:
from charset_normalizer import detect
from keras.models import Model as Model_
from keras.models import Sequential
from keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation, Dense
from PIL import Image
import numpy as np
from matplotlib import pyplot
from typing import Optional, Union
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
import pandas as pd
from datetime import datetime
import warnings

Vectorization model
---

In [ ]:
class VGGFace(Sequential):
    def __init__(self):
        super(VGGFace, self).__init__() 
                                         
        self.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))  
        self.add(Convolution2D(64, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(64, (3, 3), activation='relu'))
        self.add(MaxPooling2D((2,2), strides=(2,2)))
        
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(128, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(128, (3, 3), activation='relu'))
        self.add(MaxPooling2D((2,2), strides=(2,2)))
        
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(256, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(256, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(256, (3, 3), activation='relu'))
        self.add(MaxPooling2D((2,2), strides=(2,2)))
        
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(512, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(512, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(512, (3, 3), activation='relu'))
        self.add(MaxPooling2D((2,2), strides=(2,2)))
        
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(512, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(512, (3, 3), activation='relu'))
        self.add(ZeroPadding2D((1,1)))
        self.add(Convolution2D(512, (3, 3), activation='relu'))
        self.add(MaxPooling2D((2,2), strides=(2,2)))
        
        self.add(Convolution2D(4096, (7, 7), activation='relu'))
        self.add(Dropout(0.5))
        self.add(Convolution2D(4096, (1, 1), activation='relu'))
        self.add(Dropout(0.5))
        self.add(Convolution2D(2622, (1, 1)))
        self.add(Flatten())
        self.add(Activation('softmax'))

Photo -> Login Model
---

In [ ]:
class Model:
    image_size = (224, 224)
    model = None

    def __init__(self, model = None):
        self.model_ready = False
                                  
        if not Model.model and not model:
            Model.model = self.VGGFace_model()
        elif model:
            Model.model = model
        else:
            self.model_ready = True

    def VGGFace_model(self):
        vgg_model = VGGFace()
        vgg_model.load_weights('../misc/vgg_face_weights.h5')
        self.model_ready = True
        vgg_face_descriptor = Model_(inputs=vgg_model.layers[0].input, outputs=vgg_model.layers[-2].output)

        return vgg_face_descriptor
      
    @classmethod
    def detect_face(cls, image: Union[str, np.array]) -> np.array:
        if type(image) is str:
            image = pyplot.imread(image)

        detector = MTCNN()
        faces = detector.detect_faces(image)

        x1, y1, width, height = faces[0]['box']
        x2, y2 = x1 + width, y1 + height

        face = image[y1:y2, x1:x2]

        face_image = Image.fromarray(face)
        face_array = np.asarray(face_image)

        return face_array
        

    def get_face_embedding(self, image: Union[str, np.array]) -> np.array:
        if type(image) is str:
            image = pyplot.imread(image)
        
        img = Image.fromarray(image)
        image = np.asarray(img.resize(self.image_size)).reshape(-1, 224, 224, 3)
        embedding = self.model.predict(image)[0,:]

        return embedding

    def compare(self, image1: np.array, image2: np.array) -> int:
        if not self.model_ready:
            raise RuntimeError("Модель еще не обучена")

        if cosine(image1, image2) < 0.45:
            return 1
        return 0
    
    def recognize_from_db(self, image: Union[str, np.array], db) -> str:
        face = [self.get_face_embedding(self.detect_face(image))]
        for i, embedding in enumerate(db.get_all()):
            if self.compare(face, embedding):
                return db.df.login[db.df.index == i].item()
        
        return None


DataBase
---

In [ ]:
class DataBase:
    BASE_DIR =  './logins.pckl'
    columns = ['login', 'embedding', 'last_update']

    def __init__(self) -> None:
        try:
            self.df = pd.read_pickle(self.BASE_DIR)
            self.df.reset_index(drop=True, inplace=True)
        except FileNotFoundError:
            self.df = pd.DataFrame(columns=['login', 'embedding', 'last_update'])
            self.save()

    def add_new_face(self, login: str, embedding: np.array):
        time = datetime.isoformat(datetime.now(), timespec = 'seconds', sep=' ')
        if login in set(self.df.login):
            warnings.warn('Login already in database. Use update_face method to add new user.')
            return 
          
        new_row = pd.DataFrame([[login, [embedding], time]], columns=self.columns)
        self.df = pd.concat([self.df, new_row])
        self.df.reset_index(drop=True, inplace=True)
        self.save()

    def update_face(self, login: str, embedding: np.array):
        if login not in set(self.df.login):
            warnings.warn('No such login in database. Use add_new_face method to update user face.')
            return
        
        self.df = self.df[self.df.login != login]
        self.add_new_face(login, embedding)

    def delete_user(self, login):
        self.df = self.df[self.df.login != login]
        self.save()

    def get(self, login: str) -> np.array:
        return self.df.embedding[self.df.login == login]

    def get_all(self):
        return self.df.embedding

    def count(self) -> int:
        return self.df.shape[0]

    def clear(self):
        self.df = pd.DataFrame(columns=['login', 'embedding', 'last_update'])
        self.save()

    def save(self):
        self.df.to_pickle(self.BASE_DIR)


API
---

In [ ]:
def register_user(login: str, image: Union[str, np.array])-> None:
    model = Model()
    bd = DataBase()
    bd.add_new_face(login, model.get_face_embedding(model.detect_face(image)))

def update_user_info(login: str, image: Union[str, np.array]) -> None:
    model = Model()
    bd = DataBase()
    bd.update_face(login, model.get_face_embedding(model.detect_face(image)))

def delete_user_data(login: str) -> None:
    bd = DataBase()
    bd.delete_user(login)

def detect_user(image: Union[str, np.array]) -> str:
    model = Model()
    bd = DataBase()
    login = model.recognize_from_db(image, bd)
    return login

def compare_users(image1, image2):
    model = Model()

    face1 = model.detect_face(image1)
    face2 = model.detect_face(image2)

    embedding1 = model.get_face_embedding(face1)
    embedding2 = model.get_face_embedding(face2)

    same = model.compare(embedding1, embedding2)
    return same

def database_stats():
    db = DataBase()
    return db.count()

def database_clear():
    db = DataBase()
    db.clear()

Testing
---

In [ ]:
n1 = Model()

In [ ]:
img = pyplot.imread("../data-samples/dzh.jpg")
img2 =  pyplot.imread("../data-samples/jackwhite.jpg")
img3 =  pyplot.imread("../data-samples/jackwhite2.jpg")
img4 =  pyplot.imread("../data-samples/Jack_White_Ottawa.jpg")
img5 = pyplot.imread("../data-samples/dzhigurda.jpg")
img6 = pyplot.imread("../data-samples/potter.jpg")
img7 = pyplot.imread("../data-samples/potter3.jpg")
img8 = pyplot.imread("../data-samples/potter1.jpg")
img9 = pyplot.imread("../data-samples/dzhigurda2.jpg")
img10 = pyplot.imread("../data-samples/dzhigurda3.jpg")
img11 =  pyplot.imread("../data-samples/Jack-White.jpg")
img12 = pyplot.imread("../data-samples/potter4.jpg")

In [ ]:
class1_image = pyplot.imread("../data-samples/dzhigurda1.jpg")
class2_image = pyplot.imread("../data-samples/potter2.jpg")
class3_image = pyplot.imread("../data-samples/jackwhite3.jpg")

In [ ]:
login1 = 'gjiga'
login2 = 'potter'
login3 = 'jackwhite'

In [ ]:
register_user(login1, class1_image)
register_user(login2, class2_image)
register_user(login3, class3_image)

In [ ]:
print("djiga:", detect_user(img))
print("djiga:", detect_user(img5))
print("djiga:", detect_user(img9))
print("djiga:", detect_user(img10))

In [ ]:
print("potter:", detect_user(img6))
print("potter:", detect_user(img7))
print("potter:", detect_user(img8))
print("potter:", detect_user(img12))

In [ ]:
print("jack:", detect_user(img2))
print("jack:", detect_user(img3))
print("jack:", detect_user(img4))
print("jack:", detect_user(img11))

In [ ]:
print(detect_user(class1_image))
print(detect_user(class2_image))
print(detect_user(class3_image))

In [ ]:
print(compare_users(class1_image, class1_image))
print(compare_users(class1_image, class2_image))

In [ ]:
update_user_info(login1, class2_image)
update_user_info(login2, class1_image)
print("djiga:", detect_user(class1_image))
print("potter:", detect_user(class2_image))

In [ ]:
print(database_stats())

In [ ]:
delete_user_data(login1)
print(database_stats())

In [ ]:
database_clear()
print(database_stats())